In [41]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path
os.environ["TFHUB_CACHE_DIR"] = str(Path("./tf_hub_cache").resolve())
import re

In [18]:
em=None
#em = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")

In [80]:
model = keras.models.Sequential()
model.add(hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", input_shape=[], dtype=tf.string, output_shape=[50]))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(5, activation="softmax"))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [81]:
data = pd.read_csv("datasets\\training.1600000.processed.noemoticon.csv", names=["target", "ids", "date", "flag", "user", "text"], encoding="ISO-8859-1")

In [82]:
train_x, test_x, train_y, test_y = train_test_split(np.asarray(data.text), np.asarray(data.target), test_size=0.85)
print(len(train_x), len(train_y), len(test_x), len(test_y))

240000 240000 1360000 1360000


In [83]:
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, test_size=0.90)
test_x = test_x[:50000]
test_y = test_y[:50000]
print(len(val_x), len(val_y), len(test_x), len(test_y))

136000 136000 50000 50000


In [84]:
train_x

array(['@LadyHamburg  *jjjeeeehhhhaaaaa*  oh,what a night ',
       '@chackmaul Thankx!!!!  You do the same.',
       "it's a miserable, rainy day. enough is enough ", ...,
       "listening to................&quot;baby,come back to me&quot; .....i can't sleep.......its 3:10a.m....either way i don't want to sleep ",
       "I'm very proud of myself for not causing a shrieky scene at the airport last night. ",
       "@erinlouise Oh I love being a student! I don't love all the money it costs to do so though.  But if I could I'd be a professional student."],
      dtype=object)

In [85]:
train_x = np.array(list(map(lambda text: re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", text), train_x)))
val_x = np.array(list(map(lambda text: re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", text), val_x)))
test_x = np.array(list(map(lambda text: re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", text), test_x)))

In [86]:
test_x

array(['finally home its FREEZING outside today rainy gloomy amp I have to go to a graduation ceremony today today has not been a good day ',
       'New bike arrived today Slight ding on the side ', '  ', ...,
       'Half day But busy afternoon and evening ', '  glad to hear it ',
       '  hehe do you do it twice i do hahaha '], dtype='<U162')

In [94]:
with tf.device("/CPU:0"):
    model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=5, batch_size=64, workers=7)

Train on 240000 samples, validate on 136000 samples
Epoch 1/5
  4224/240000 [..............................] - ETA: 5:10 - loss: 0.5861 - accuracy: 0.6853

KeyboardInterrupt: 